In [1]:
from pprint import pprint

import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff.forcefield import ForceField
from openff.toolkit.utils import get_data_file_path
from openff.units import unit
from openmm import app
from openmm import unit as openmm_unit

from openff.interchange.components.interchange import Interchange

In [2]:
# Load a PDB file packaged with the OpenFF Toolkit
pdbfile = app.PDBFile(
    get_data_file_path("systems/packmol_boxes/propane_methane_butanol_0.2_0.3_0.5.pdb")
)

In [3]:
# The OpenFF Topology currently requires that all molecular representations have
# chemiformatics data not present in a PDB file, including bond orders and
# stereochemistry. Therefore, Topology.from_openmm() requiures a list of
# Molecule objects with this data to be passed through. For more, see
# https://open-forcefield-toolkit.readthedocs.io/en/latest/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_openmm)

molecules = [Molecule.from_smiles(smi) for smi in ["CCC", "C", "CCCCO"]]
topology = Topology.from_openmm(pdbfile.topology, unique_molecules=molecules)

In [4]:
# Load in a mainline OpenFF force field
sage = ForceField("openff-2.0.0.offxml")

In [5]:
# Create an Interchange object
out = Interchange.from_smirnoff(force_field=sage, topology=topology)

In [6]:
# The OpenFF Topology represents a chemical graph without explicit positions, so
# set the positions of the Interchange object with the positions in the PDB fiile
out.positions = pdbfile.positions

In [7]:
# Topology.from_openmm(), however, reads the periodic vectors from the PDB file and
# Interchnage.from_smirnoff() uses these to set the .box attribute, so we don't need
# to set it. But we can verify that they are equal
assert np.allclose(
    out.box.m_as(unit.nanometer),
    pdbfile.topology.getPeriodicBoxVectors().value_in_unit(openmm_unit.nanometer),
)

In [8]:
# The Interchange package includes a module for obtaining single-point energies of
# Interchange objects by calling out to molecular mechanics engines. Here we query,
# inspect, and compare the energies obtained via OpenMM and GROMACS
from openff.interchange.drivers import (
    get_gromacs_energies,
    get_lammps_energies,
    get_openmm_energies,
)

gromacs_energies = get_gromacs_energies(out)
openmm_energies = get_openmm_energies(out)
lammps_energies = get_lammps_energies(out)

In [9]:
print(openmm_energies)

Energies:

Bond:          		42.18247985839844 kJ / mol
Angle:         		17956.1171875 kJ / mol
Torsion:       		1310.050048828125 kJ / mol
Nonbonded:     		None
vdW:           		-7173.355622467902 kJ / mol
Electrostatics:		-2839.989448849101 kJ / mol



In [10]:
print(gromacs_energies)

Energies:

Bond:          		42.18257141113281 kJ / mol
Angle:         		17956.171875 kJ / mol
Torsion:       		1310.049072265625 kJ / mol
Nonbonded:     		None
vdW:           		-7173.39599609375 kJ / mol
Electrostatics:		-2841.615219116211 kJ / mol



In [11]:
print(lammps_energies)

Energies:

Bond:          		11.541868 kcalorie_per_mole
Angle:         		4291.6175 kcalorie_per_mole
Torsion:       		313.10955 kcalorie_per_mole
Nonbonded:     		None
vdW:           		-1832.22855 kcalorie_per_mole
Electrostatics:		-678.79 kcalorie_per_mole



In [12]:
pprint(openmm_energies - gromacs_energies)

{'Angle': <Quantity(-0.0546875, 'kilojoule / mole')>,
 'Bond': <Quantity(-9.15527344e-05, 'kilojoule / mole')>,
 'Electrostatics': <Quantity(1.62577027, 'kilojoule / mole')>,
 'Torsion': <Quantity(0.0009765625, 'kilojoule / mole')>,
 'vdW': <Quantity(0.0403736258, 'kilojoule / mole')>}


In [13]:
pprint(openmm_energies - lammps_energies)

{'Angle': <Quantity(-0.0104325, 'kilojoule / mole')>,
 'Bond': <Quantity(-6.10869585, 'kilojoule / mole')>,
 'Electrostatics': <Quantity(0.0679111509, 'kilojoule / mole')>,
 'Torsion': <Quantity(-0.000308371875, 'kilojoule / mole')>,
 'vdW': <Quantity(492.688631, 'kilojoule / mole')>}
